In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
plt.style.use('ggplot')

from sklearn.model_selection import train_test_split

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.svm import SVC
import statsmodels.api as sm

# Question 3: Y/N Hourly Outage

## Pre-processing data

**Importing & Cleaning DF**

In [2]:
DF3_yn_hrly_outage = pd.read_csv('model_dfs/DF3_yn_hrly_outage.csv') 
# Limit data for memory purposes for now. In HW 9, the full dataframe had 1489 x 26 observations. 
# As a result, selected 1 feeders 8784 x 58 features
# Tried n = 1500, 2000 but wouldn't run w/in 5 min
DF3_yn_hrly_outage = DF3_yn_hrly_outage.sample(n = 1500, random_state = 2021)
DF3_yn_hrly_outage = DF3_yn_hrly_outage.drop(columns = ['Unnamed: 0'])
DF3_yn_hrly_outage['hour'] = pd.to_datetime(DF3_yn_hrly_outage['hour'])
print(DF3_yn_hrly_outage.shape)
DF3_yn_hrly_outage.to_csv('model_dfs/DF3_famia_1500_sample')
DF3_yn_hrly_outage.head(5)

(1500, 5)


,feeder_name,Voltage,BUSINESS HUB,hour,outage
8330,famia,11KV,ILE-IFE,2020-12-13 02:00:00,1.0
3556,famia,11KV,ILE-IFE,2020-05-28 04:00:00,1.0
8652,famia,11KV,ILE-IFE,2020-12-26 12:00:00,0.0
2625,famia,11KV,ILE-IFE,2020-04-19 09:00:00,1.0
395,famia,11KV,ILE-IFE,2020-01-17 11:00:00,1.0


In [3]:
# Read in grid char data
grid_char = pd.read_csv('merge_in/TCN_DATA USE.csv')
grid_char = grid_char.drop(columns = ['Unnamed: 0'])
DF3_yn_hrly_outage['date'] = DF3_yn_hrly_outage['hour'].dt.date
grid_char['Date'] = pd.to_datetime(grid_char['Date']).dt.date

# Merge w/ grid/power system data
DF3_yn_hrly_outage = pd.merge(DF3_yn_hrly_outage, grid_char, how = 'left', 
                              left_on = ['date'], right_on = ['Date'])

DF3_yn_hrly_outage = DF3_yn_hrly_outage.drop(columns = ['Date', 'date'])

# # Drop rows with null values
#DF3_yn_hrly_outage.isnull().sum()
DF3_yn_hrly_outage = DF3_yn_hrly_outage.dropna().reset_index(drop=True)

In [4]:
# Read in hourly weather data
hrly_weather = pd.read_csv('merge_in/near_weather_all_hours.csv')
hrly_weather['hour'] = pd.to_datetime(hrly_weather['hour'])

# Merge w/ hourly weather data
DF3_yn_hrly_outage = pd.merge(DF3_yn_hrly_outage, hrly_weather, how = 'left', 
                             left_on = ['hour'], right_on = ['hour'])

pd.set_option('display.max_columns', 200)
DF3_yn_hrly_outage.head(5)

DF3_yn_hrly_outage = DF3_yn_hrly_outage.dropna().reset_index(drop=True)

In [5]:
# Create columns for month/dow/hod
DF3_yn_hrly_outage['month'] = DF3_yn_hrly_outage['hour'].dt.month
DF3_yn_hrly_outage['dow'] = DF3_yn_hrly_outage.loc[:,'hour'].apply(lambda x: x.weekday())
DF3_yn_hrly_outage['hod'] = DF3_yn_hrly_outage['hour'].dt.hour
months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
days = ['mon', 'tues', 'wed', 'thurs', 'fri', 'sat', 'sun']
mo_dict = dict(zip(np.arange(1, 13), months))
day_dict = dict(zip(np.arange(0, 7), days))
DF3_yn_hrly_outage = DF3_yn_hrly_outage.join(pd.get_dummies(DF3_yn_hrly_outage['month'])).rename(columns = mo_dict)
DF3_yn_hrly_outage = DF3_yn_hrly_outage.join(pd.get_dummies(DF3_yn_hrly_outage['dow'])).rename(columns = day_dict)
DF3_yn_hrly_outage = DF3_yn_hrly_outage.join(pd.get_dummies(DF3_yn_hrly_outage['hod'], prefix='hr'))

# One-hot encode feeders, voltage, & business hub
# COMMENT THIS OUT FOR SINGLE FEEDER MODEL
# DF3_yn_hrly_outage = DF3_yn_hrly_outage.join(pd.get_dummies(DF3_yn_hrly_outage['BUSINESS HUB']))
# DF3_yn_hrly_outage = DF3_yn_hrly_outage.join(pd.get_dummies(DF3_yn_hrly_outage['Voltage']))
# DF3_yn_hrly_outage = DF3_yn_hrly_outage.join(pd.get_dummies(DF3_yn_hrly_outage['feeder_name']))

DF3_yn_hrly_outage = DF3_yn_hrly_outage.drop(columns = ['month', 'hod', 'dow', 'feeder_name', 'Voltage', 'BUSINESS HUB'])

pd.set_option('display.max_columns', 200)
DF3_yn_hrly_outage.head(5)

,hour,outage,Peak Generation (MW),Lowest Generation (MW),Energy Recorded(Mwh),Generation at 6:00hrs (MW),Highest System Frequency (Hz),Lowest System Frequency (Hz),Highest Voltage Recorded (kV),Lowest Voltage Recorded (kV),atmosphericpressure (kPa),precipitation (mm),radiation (W/m2),relativehumidity (-),temperature (degrees Celsius),winddirection (degrees),windgusts (m/s),windspeed (m/s),jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec,mon,tues,wed,thurs,fri,sat,sun,hr_0,hr_1,hr_2,hr_3,hr_4,hr_5,hr_6,hr_7,hr_8,hr_9,hr_10,hr_11,hr_12,hr_13,hr_14,hr_15,hr_16,hr_17,hr_18,hr_19,hr_20,hr_21,hr_22,hr_23
0,2020-12-13 02:00:00,1.0,5209.0,3771.6,109139.92,4482.55,50.509,49.61,350.0,300.0,98.91,0.0,0,0.831,26.0,261.0,1.45,1.04,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2020-05-28 04:00:00,1.0,5114.2,3266.3,94565.93,3470.90,50.650,49.74,352.0,300.0,99.00,0.0,0,0.844,24.0,290.0,1.10,0.59,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2020-12-26 12:00:00,0.0,5223.5,3406.7,101791.08,3630.70,50.420,49.42,352.0,302.0,98.90,0.0,580,0.405,30.2,295.0,2.30,0.89,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,2020-04-19 09:00:00,1.0,5012.3,3650.0,101433.79,3866.70,50.570,49.88,353.0,300.0,99.27,0.0,226,0.823,27.9,240.0,1.54,0.74,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2020-01-17 11:00:00,1.0,4345.6,2278.7,84021.44,2931.00,50.950,49.78,350.0,300.0,99.26,0.0,539,0.690,28.8,14.0,2.78,1.13,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
# Check how common outages are
DF3_yn_hrly_outage['outage'].sum()/len(DF3_yn_hrly_outage['outage'])

0.6213063763608087

**Split data into train & test**

In [7]:
# Split into train/test
features = DF3_yn_hrly_outage.drop(columns = ['hour', 'outage'])
target = DF3_yn_hrly_outage['outage']

# make the test/train split
X, X_test, y, y_test = train_test_split(features, target, test_size = 0.2, random_state = 2021)

# make the train/validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=2021)

## Model 1: Boosting

In [8]:
gb_tree = GradientBoostingClassifier(random_state = 2021)

param_dist = {'n_estimators': randint(50, 300), 
              'min_samples_split': randint(2, 50), 
              'max_depth': randint(2, 50)} 

rnd_gb_search = RandomizedSearchCV(gb_tree, param_distributions=param_dist, 
                                   cv=5, n_iter=10, random_state = 2021)

# Takes ~1 min with 1000 observations, cv = 5, n_iter = 10
# Tried with 15 n_iter, up to 500 n_estimators, took >5 min to run and val accuracy actually went down, so 
# switched back to less computationally expensive options
rnd_gb_search.fit(X_train, y_train)

print(rnd_gb_search.best_params_)

{'max_depth': 31, 'min_samples_split': 26, 'n_estimators': 190}


In [9]:
# Fit model on full training dataset and get test score
# Fit model on optimized parameters
# Takes about 5 minutes to run
gb_tree = GradientBoostingClassifier(random_state = 2021, n_estimators = 190, 
                                     min_samples_split = 26, max_depth = 31)
gb_tree.fit(X_train, y_train)

gb_train_score = gb_tree.score(X_train, y_train)
gb_val_score = gb_tree.score(X_val, y_val)

print('Train Accuracy Score: ', gb_train_score)
print('Validation Accuracy Score: ', gb_val_score)

Train Accuracy Score:  1.0
Validation Accuracy Score:  0.6264591439688716


In [10]:
def feature_importance_df(tree): 
    feature_importance = tree.feature_importances_
    feat_df = pd.DataFrame({'feature':X_train.columns, 'importance':feature_importance})
    feat_df = feat_df.sort_values(by='importance', ascending=False)
    #Commented out plot since kills kernel, instead just get df
    #plt.barh(width=feat_df['importance'], y=feat_df['feature'])
    return feat_df

In [11]:
y_pred = gb_tree.predict(X_val)

print(confusion_matrix(y_val,y_pred)) 
# TN FP
# FN TP
TN, FP, FN, TP = confusion_matrix(y_val,y_pred).ravel().ravel()
precision = TP/(TP + FP)
recall = TP/(TP + FN)
print('precision: ', precision)
print('recall: ', recall)

[[ 27  64]
 [ 32 134]]
precision:  0.6767676767676768
recall:  0.8072289156626506


In [12]:
gb_importance_df = feature_importance_df(gb_tree)
gb_importance_df

,feature,importance
8,atmosphericpressure (kPa),0.089490
14,windgusts (m/s),0.083638
15,windspeed (m/s),0.077855
0,Peak Generation (MW),0.077305
11,relativehumidity (-),0.065807
5,Lowest System Frequency (Hz),0.064121
13,winddirection (degrees),0.054279
12,temperature (degrees Celsius),0.050826
2,Energy Recorded(Mwh),0.050047
4,Highest System Frequency (Hz),0.045398


## Model 2: Random Forest

In [13]:
rf_tree = RandomForestClassifier(random_state = 2021)

param_dist = {'n_estimators': randint(50, 300), 
              'min_samples_split': randint(2, 50), 
              'max_depth': randint(2, 50)} 

rnd_rf_search = RandomizedSearchCV(rf_tree, param_distributions=param_dist, 
                                   cv=5, n_iter=10, random_state = 2021)

# Takes ~1 min with 1000 observations, cv = 5, n_iter = 10
# Kernel dies under same conditions with 1500, 2000 observations
rnd_rf_search.fit(X_train, y_train)

print(rnd_rf_search.best_params_)

{'max_depth': 8, 'min_samples_split': 40, 'n_estimators': 120}


In [14]:
# Fit model on full training dataset and get test score
# Fit model on optimized parameters
rf_tree = RandomForestClassifier(random_state = 2021, n_estimators = 120, 
                                     min_samples_split = 40, max_depth = 8)
rf_tree.fit(X_train, y_train)

rf_train_score = rf_tree.score(X_train, y_train)
rf_val_score = rf_tree.score(X_val, y_val)

print('Train Accuracy Score: ', rf_train_score)
print('Validation Accuracy Score: ', rf_val_score)

Train Accuracy Score:  0.6770428015564203
Validation Accuracy Score:  0.6614785992217899


In [15]:
y_pred = rf_tree.predict(X_val)

print(confusion_matrix(y_val,y_pred)) 
# TN FP
# FN TP
TN, FP, FN, TP = confusion_matrix(y_val,y_pred).ravel().ravel()
precision = TP/(TP + FP)
recall = TP/(TP + FN)
print('precision: ', precision)
print('recall: ', recall)

[[  4  87]
 [  0 166]]
precision:  0.6561264822134387
recall:  1.0


In [16]:
gb_importance_df = feature_importance_df(rf_tree)
gb_importance_df

,feature,importance
2,Energy Recorded(Mwh),0.075193
0,Peak Generation (MW),0.064010
15,windspeed (m/s),0.059160
3,Generation at 6:00hrs (MW),0.057016
8,atmosphericpressure (kPa),0.054437
12,temperature (degrees Celsius),0.054003
14,windgusts (m/s),0.052892
13,winddirection (degrees),0.052258
11,relativehumidity (-),0.051723
10,radiation (W/m2),0.051154


## Model 3: SVM

In [17]:
svclassifier = SVC(random_state = 2021, C = 1.0)

param_dist = {'kernel': ('rbf', 'sigmoid'), 
              'C': [1, 10, 100]} 

rnd_svm_search = RandomizedSearchCV(svclassifier, param_distributions = param_dist, 
                                   cv=5, n_iter=10, random_state = 2021)

rnd_svm_search.fit(X_train, y_train.values.ravel())

print(rnd_svm_search.best_params_)

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


{'kernel': 'rbf', 'C': 1}


In [18]:
# Fit model on full training dataset and get test score
# Fit model on optimized parameters
svclassifier = SVC(random_state = 2021, kernel = 'rbf', C = 1)

svclassifier.fit(X_train, y_train.values.ravel())
y_pred = svclassifier.predict(X_val)

svm_train_score = svclassifier.score(X_train, y_train)
svm_val_score = svclassifier.score(X_val, y_val)

print('Train Accuracy Score: ', svm_train_score)
print('Validation Accuracy Score: ', svm_val_score)

Train Accuracy Score:  0.6251621271076524
Validation Accuracy Score:  0.6459143968871596


In [19]:
print(confusion_matrix(y_val,y_pred)) 
# TN FP
# FN TP
TN, FP, FN, TP = confusion_matrix(y_val,y_pred).ravel().ravel()
precision = TP/(TP + FP)
recall = TP/(TP + FN)
print('precision: ', precision)
print('recall: ', recall)

[[  0  91]
 [  0 166]]
precision:  0.6459143968871596
recall:  1.0


## Model 4: Logistic Regression

In [20]:
# Commented out since logistic regression does not converge to optimal solution
#y_train_new = logit_fit.params
#logit = sm.Logit(y_train_new,X_train)
# logit = sm.Logit(y_train,X_train)
# #default maxiter=35, should converge quickly but failing to converge even when set max iterations very high (5000)
# logit_fit = logit.fit(maxiter = 100)
# logit_fit.params

## Training Selected Model & Retrieving Test MSE

In [21]:
# Fit model on full training dataset and get test score
# Fit model on optimized parameters
rf_tree = RandomForestClassifier(random_state = 2021, n_estimators = 120, 
                                     min_samples_split = 40, max_depth = 8)
rf_tree.fit(X, y)

rf_train_score = rf_tree.score(X, y)
rf_val_score = rf_tree.score(X_test, y_test)

print('Train Accuracy Score: ', rf_train_score)
print('Test Accuracy Score: ', rf_val_score)

Train Accuracy Score:  0.6653696498054474
Test Accuracy Score:  0.5813953488372093


In [22]:
y_pred = rf_tree.predict(X_test)
print(confusion_matrix(y_test,y_pred)) 
# TN FP
# FN TP
TN, FP, FN, TP = confusion_matrix(y_test,y_pred).ravel().ravel()
precision = TP/(TP + FP)
recall = TP/(TP + FN)
print('precision: ', precision)
print('recall: ', recall)

[[  4 103]
 [  5 146]]
precision:  0.5863453815261044
recall:  0.9668874172185431


In [23]:
gb_importance_df = feature_importance_df(rf_tree)
gb_importance_df

,feature,importance
15,windspeed (m/s),0.069697
0,Peak Generation (MW),0.066135
12,temperature (degrees Celsius),0.063926
5,Lowest System Frequency (Hz),0.062629
2,Energy Recorded(Mwh),0.062412
11,relativehumidity (-),0.056835
3,Generation at 6:00hrs (MW),0.056108
14,windgusts (m/s),0.050805
6,Highest Voltage Recorded (kV),0.046841
8,atmosphericpressure (kPa),0.044572
